# FIFA 21 cleaning project

This is a simple cleaning project concerning a dataset of EA Sports' FIFA21 which was scraped from sofifa.com and [ obtained through kaggle](https://www.kaggle.com/yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring). It was performed with Anaconda python 3.8. The first 5 lines of the dataset as well as the data types attributed to it by Pandas are shown below.

In [1]:
import pandas as pd
import re

fifaData = pd.read_csv('fifa21 raw data v2.csv')
pd.set_option("display.max_rows",100)
pd.set_option("display.max_columns",100)
print(fifaData.head(5))
print('\n')
print(fifaData.dtypes)

       ID               Name                      LongName  \
0  158023           L. Messi                  Lionel Messi   
1   20801  Cristiano Ronaldo  C. Ronaldo dos Santos Aveiro   
2  200389           J. Oblak                     Jan Oblak   
3  192985       K. De Bruyne               Kevin De Bruyne   
4  190871          Neymar Jr    Neymar da Silva Santos Jr.   

                                           photoUrl  \
0  https://cdn.sofifa.com/players/158/023/21_60.png   
1  https://cdn.sofifa.com/players/020/801/21_60.png   
2  https://cdn.sofifa.com/players/200/389/21_60.png   
3  https://cdn.sofifa.com/players/192/985/21_60.png   
4  https://cdn.sofifa.com/players/190/871/21_60.png   

                                           playerUrl Nationality  Age  ↓OVA  \
0  http://sofifa.com/player/158023/lionel-messi/2...   Argentina   33    93   
1  http://sofifa.com/player/20801/c-ronaldo-dos-s...    Portugal   35    92   
2  http://sofifa.com/player/200389/jan-oblak/210006/    Slo

C:\Users\kostas\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (76) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


I first removed the photoUrl and playerUrl columns since it is unlikely that these web links would play any role in an analysis.

In [2]:
fifaData.drop(['photoUrl','playerUrl'],axis=1,inplace=True)

Then I removed unnecessary newline characters from the Club column.

In [3]:
cleanedClub = fifaData['Club'].str.extract(r'([^\n]+)',expand=False)
fifaData['Club'] = cleanedClub

Subsequently I removed the star (★) characters from the IR, W/F and SM columns, thereby retaining only the numerical rating. Then the columns were converted to numerical form (int64).

In [4]:
cleanedIR = fifaData['IR'].str.extract(r'(\d)',expand=False)
cleanedWF = fifaData['W/F'].str.extract(r'(\d)',expand=False)
cleanedSM = fifaData['SM'].str.extract(r'(\d)',expand=False)
fifaData['IR'] = pd.to_numeric(cleanedIR)
fifaData['W/F'] = pd.to_numeric(cleanedWF)
fifaData['SM'] = pd.to_numeric(cleanedSM)

I also converted Height and Weight to a numerical type (int64) and renamed the columns to specify the unit of measurement.

In [5]:
height = fifaData['Height'].str.extract(r'(\d+)',expand=False)
weight = fifaData['Weight'].str.extract(r'(\d+)',expand=False)
fifaData['Height']=pd.to_numeric(height)
fifaData['Weight']=pd.to_numeric(weight)
newNames = {'Height':'Height (cm)','Weight':'Weight (kg)'}
fifaData.rename(columns=newNames,inplace=True)

Finally I converted the Value, Wage, Release Clause and Hits columns to numerical type. These columns contained character indicators for thousands (K) and millions (M) thus necessitating the multiplication of the values by the appropriate multiplier before conversion. This was achieved with a custom function. All four new dtypes are float64.

In [6]:
def amountMultiplier(item):
    if pd.isna(item) == True:
        return item
    else:
        item = str(item)
        
    if item[-1] == 'M':
        item = float(re.search(r'[\d.]+',item).group())
        item  = item*1000000
        return item
    elif item[-1] == 'K':
        item = float(re.search(r'[\d.]+',item).group())
        item = item*1000
        return item
    elif item[-1].isalpha() == True:
        raise Exception('Unidentified multiplier')
    else:
        item = float(re.search(r'[\d.]+',item).group())
        return item

cleanedValue = fifaData['Value'].apply(amountMultiplier)
fifaData['Value'] = cleanedValue
cleanedWage = fifaData['Wage'].apply(amountMultiplier)
fifaData['Wage'] = cleanedWage
cleanedRC = fifaData['Release Clause'].apply(amountMultiplier)
fifaData['Release Clause'] = cleanedRC
cleanedHits = fifaData['Hits'].apply(amountMultiplier)
fifaData['Hits'] = cleanedHits

A sample of 30 rows of the cleaned dataset is shown below and the dataset is exported to a csv format.

In [7]:
print(fifaData.sample(30))
fifaData.to_csv('FIFA21_cleaned.csv')

           ID             Name                       LongName  Nationality  \
15145  245628         A. Abqar               Abdelkabir Abqar      Morocco   
12890  259019   Renan Oliveira     Renan Abner C. de Oliveira       Brazil   
12761  238786        O. Käuper                     Ole Käuper      Germany   
2980   139313         C. Fuchs                Christian Fuchs      Austria   
3365   234857      I. Oblyakov                  Ivan Oblyakov       Russia   
18395  257024       M. Marleku                   Mark Marleku       Kosovo   
11240  248702        J. Castro                    Juan Castro       Mexico   
1587   214572        J. Angulo                    Juan Angulo     Colombia   
4378   231391    João Carvalho  João António Antunes Carvalho     Portugal   
990    185195        O. Ighalo                   Odion Ighalo      Nigeria   
9048   189836         J. Mills                   Joseph Mills      England   
13393  255595        Y. Senuma                    Yuji Senuma   